# Train an ML Model That Finds Cryptic Pockets Within MD Trajectories.
The training dataset consits of crystal structures from the extended Cryptosite database.  There are several structures for each protein in the database.  One structures is "holo"; i.e. it has a cryptic pocket.  The other structures are "apo"; i.e. they (hopefully) don't have the pocket.  The data is meant to mimic an MD trajectory in which some frames have the pocket and others don't.

It is assumed that the only holo structure is the one found the Cryptosite database.  This assumption isn't always true; i.e. some of the (supposedly) "apo" structures may have cryptic pockets that are partially or fully open.  This introduces error into the database; the database is expected to be useful despite the error.

## Initialize the dataset.
The notebook cell below creates the dataframe `df_train` that stores training data for ML.  It contains a line for each residue of each protein in the extended database.  Each line contains the following:
* Info about the protein and residue.
* Info about the "crypto_apo" protein.  This is the apo protein from the Cryptosite database with the same sequence as the protein.
* Info about the holo protein from the Cryptosite database.
* Misc. features for ML, which are added later in the notebook.

In [1]:
import glob
import os
import shutil

import pandas as pd

from align import align_res_nums
from get_concavity_score import get_concavity_score


# Delete old output
conc_output_dir = "train_set_conc"
if os.path.isdir(conc_output_dir):
    shutil.rmtree(conc_output_dir)
os.mkdir(conc_output_dir)

# Initialize the training set data as a list.  Later, the code will convert this list to a DataFrame.
# This is faster than iteratively creating a DataFrame.  The columns are:
# "prot_id", "chain_id", "resnum", "holo_prot_id", "holo_chain_id", "holo_resnum",
# "crypto_apo_prot_id","crypto_apo_chain_id", "crypto_apo_resnum", "is_site"
#
# Start by adding the data from the apo and holo proteins in the original cryptosite database.  This is
# necessary because later parts of the code require that all apo and holo structures in the original
# Cryptosite database are present.
#
# Next, add the proteins from the extended database.  Add a maximum of 50 extended-database proteins per
# Cryptosite apo protein.  This is done so that the database is less biased towards proteins that are
# crystallized frequently.  Some bias still exists because not all proteins have 50 structures.

train_set_as_list = []
df_cryptosite_orig = pd.read_csv("../generate_training_database/gen_crypto_database/cryptosite_database.csv")
for index, row in df_cryptosite_orig.iterrows():
    # Add cryptosite holo.
    train_set_as_list.append([row["holo_pdb_id"].lower(), row["holo_chain"], row["holo_resnum"],
                              row["holo_pdb_id"].lower(), row["holo_chain"], row["holo_resnum"],
                              row["apo_pdb_id"].lower(), row["apo_chain"], row["apo_resnum"],
                              row["is_cryptic"]])
    # Add cryptosite apo.  By definition is_cryptic=False for all apo residues.
    train_set_as_list.append([row["apo_pdb_id"].lower(), row["apo_chain"], row["apo_resnum"],
                              row["holo_pdb_id"].lower(), row["holo_chain"], row["holo_resnum"],
                              row["apo_pdb_id"].lower(), row["apo_chain"], row["apo_resnum"],
                              False])

# Add the extended Cryptosite database to train_set_as_list.
cryptosite_db_list_loc = ("../generate_training_database/gen_crypto_database/pdbs_and_ligands.csv")
with open(cryptosite_db_list_loc, "r") as cryptosite_db_list_file:
    cryptosite_db_list_lines = cryptosite_db_list_file.readlines()

dict_holo_chains = {}
for line in cryptosite_db_list_lines[1:]:
    holo_pdb_id = line[7:11].lower()
    holo_chain_id = line[12]
    dict_holo_chains[holo_pdb_id] = holo_chain_id

dict_crypto_apo_chains = {}
for line in cryptosite_db_list_lines[1:]:
    apo_pdb_id = line[0:4].lower()
    apo_chain_id = line[5]
    dict_crypto_apo_chains[apo_pdb_id] = apo_chain_id

dict_crypto_apo_to_holo = {}
for line in cryptosite_db_list_lines[1:]:
    crypto_apo_pdb_id = line[0:4].lower()
    holo_pdb_id = line[7:11].lower()
    dict_crypto_apo_to_holo[crypto_apo_pdb_id] = holo_pdb_id

num_passed = 0 # Useful for estimating how fast the code is running
for apo_list_loc in sorted(glob.glob("../generate_training_database/ext_crypto_database/extended_db/*good*")):
    crypto_apo_pdb_id = apo_list_loc[62:66]
    crypto_apo_chain_id = dict_crypto_apo_chains[crypto_apo_pdb_id]
    holo_pdb_id = dict_crypto_apo_to_holo[crypto_apo_pdb_id]
    holo_chain_id = dict_holo_chains[holo_pdb_id]
    holo_pdb_loc = ("../generate_training_database/gen_crypto_database/holo_structures/%s.pdb" %(holo_pdb_id))
    # 1AFQ.C and 2CGA.B don't show up in each other's sequence identity clusters.  I suspect that this is because
    # part of 2CGA.B corresponds to 1AFQ.B; this is a result of 1AFQ being divided into more chains than 2CGA.
    # Regardless of if this is a correct explanation, the code must look at the cryptosite database to get the
    # crypto_apo because of this issue.
    crypto_apo_pdb_loc = ("../generate_training_database/gen_crypto_database/"
                          "apo_structures/%s.pdb" %(crypto_apo_pdb_id))
    with open(apo_list_loc, "r") as apo_list_opened:
        apo_list_lines = apo_list_opened.readlines()
    for line in apo_list_lines[1:min(len(apo_list_lines), 51)]: # Skip the header, and read only 50 lines.
        apo_pdb_id = line[0:4]
        apo_pdb_loc = ("../generate_training_database/ext_crypto_database/extended_db/"
                       "%s_download_pdbs/%s.pdb" %(crypto_apo_pdb_id, apo_pdb_id))
        apo_chain_id = line[5]
        print("num_passed", num_passed, "args", apo_pdb_loc, apo_chain_id, holo_pdb_loc, holo_chain_id)
        # apo_holo_dict maps the PDB residue numbers between the apo structure and the holo structure.
        # (The apo structure is from the extended Cryptosite database; the holo structure is from the original
        # Cryptosite database.)  This is necessary because each line of train_set_as_list will store both the
        # apo resnum and the holo resnum.
        apo_holo_dict = align_res_nums(apo_pdb_loc, apo_chain_id, holo_pdb_loc, holo_chain_id)
            
        apo_crypto_apo_dict = align_res_nums(apo_pdb_loc, apo_chain_id, crypto_apo_pdb_loc, crypto_apo_chain_id)

        for apo_resnum, holo_resnum in apo_holo_dict.items():
            crypto_apo_resnum = apo_crypto_apo_dict[apo_resnum]
            train_set_as_list.append([apo_pdb_id, apo_chain_id, apo_resnum,
                                      holo_pdb_id, holo_chain_id, holo_resnum, 
                                      crypto_apo_pdb_id, crypto_apo_chain_id, crypto_apo_resnum,
                                      False])
    num_passed += 1
df_train = pd.DataFrame(train_set_as_list, columns=["prot_id", "chain_id", "resnum",
                                                    "holo_prot_id", "holo_chain_id", "holo_resnum",
                                                    "crypto_apo_prot_id","crypto_apo_chain_id", "crypto_apo_resnum",
                                                    "is_site"])
df_train.head()

num_passed 0 args ../generate_training_database/ext_crypto_database/extended_db/1a8i_download_pdbs/5ox4.pdb A ../generate_training_database/gen_crypto_database/holo_structures/2ieg.pdb B

                         MODELLER 9.22, 2019/06/19, r11413

     PROTEIN STRUCTURE MODELLING BY SATISFACTION OF SPATIAL RESTRAINTS


                     Copyright(c) 1989-2019 Andrej Sali
                            All Rights Reserved

                             Written by A. Sali
                               with help from
              B. Webb, M.S. Madhusudhan, M-Y. Shen, G.Q. Dong,
          M.A. Marti-Renom, N. Eswar, F. Alber, M. Topf, B. Oliva,
             A. Fiser, R. Sanchez, B. Yerkovich, A. Badretdinov,
                     F. Melo, J.P. Overington, E. Feyfant
                 University of California, San Francisco, USA
                    Rockefeller University, New York, USA
                      Harvard University, Cambridge, USA
                   Imperial Cancer Research Fund, 


SALIGN_____> adding the next group to the alignment; iteration    1
num_passed 0 args ../generate_training_database/ext_crypto_database/extended_db/1a8i_download_pdbs/2qrh.pdb A ../generate_training_database/gen_crypto_database/holo_structures/2ieg.pdb B

SALIGN_____> adding the next group to the alignment; iteration    1

SALIGN_____> adding the next group to the alignment; iteration    1
num_passed 0 args ../generate_training_database/ext_crypto_database/extended_db/1a8i_download_pdbs/1xc7.pdb A ../generate_training_database/gen_crypto_database/holo_structures/2ieg.pdb B

SALIGN_____> adding the next group to the alignment; iteration    1

SALIGN_____> adding the next group to the alignment; iteration    1
num_passed 0 args ../generate_training_database/ext_crypto_database/extended_db/1a8i_download_pdbs/2qrq.pdb A ../generate_training_database/gen_crypto_database/holo_structures/2ieg.pdb B

SALIGN_____> adding the next group to the alignment; iteration    1

SALIGN_____> adding the 


SALIGN_____> adding the next group to the alignment; iteration    1

SALIGN_____> adding the next group to the alignment; iteration    1
num_passed 0 args ../generate_training_database/ext_crypto_database/extended_db/1a8i_download_pdbs/2qn3.pdb A ../generate_training_database/gen_crypto_database/holo_structures/2ieg.pdb B

SALIGN_____> adding the next group to the alignment; iteration    1

SALIGN_____> adding the next group to the alignment; iteration    1
num_passed 0 args ../generate_training_database/ext_crypto_database/extended_db/1a8i_download_pdbs/2f3p.pdb A ../generate_training_database/gen_crypto_database/holo_structures/2ieg.pdb B

SALIGN_____> adding the next group to the alignment; iteration    1

SALIGN_____> adding the next group to the alignment; iteration    1
num_passed 1 args ../generate_training_database/ext_crypto_database/extended_db/1ade_download_pdbs/1ade.pdb A ../generate_training_database/gen_crypto_database/holo_structures/1cib.pdb A

SALIGN_____> adding the 


SALIGN_____> adding the next group to the alignment; iteration    1
num_passed 3 args ../generate_training_database/ext_crypto_database/extended_db/1alv_download_pdbs/4phn.pdb B ../generate_training_database/gen_crypto_database/holo_structures/1nx3.pdb A

SALIGN_____> adding the next group to the alignment; iteration    1

SALIGN_____> adding the next group to the alignment; iteration    1
num_passed 3 args ../generate_training_database/ext_crypto_database/extended_db/1alv_download_pdbs/1alv.pdb A ../generate_training_database/gen_crypto_database/holo_structures/1nx3.pdb A

SALIGN_____> adding the next group to the alignment; iteration    1

SALIGN_____> adding the next group to the alignment; iteration    1
num_passed 3 args ../generate_training_database/ext_crypto_database/extended_db/1alv_download_pdbs/1alv.pdb B ../generate_training_database/gen_crypto_database/holo_structures/1nx3.pdb A

SALIGN_____> adding the next group to the alignment; iteration    1

SALIGN_____> adding the 


SALIGN_____> adding the next group to the alignment; iteration    1
num_passed 7 args ../generate_training_database/ext_crypto_database/extended_db/1bsq_download_pdbs/6fxb.pdb A ../generate_training_database/gen_crypto_database/holo_structures/1gx8.pdb A

SALIGN_____> adding the next group to the alignment; iteration    1

SALIGN_____> adding the next group to the alignment; iteration    1
num_passed 7 args ../generate_training_database/ext_crypto_database/extended_db/1bsq_download_pdbs/6fxb.pdb B ../generate_training_database/gen_crypto_database/holo_structures/1gx8.pdb A

SALIGN_____> adding the next group to the alignment; iteration    1

SALIGN_____> adding the next group to the alignment; iteration    1
num_passed 7 args ../generate_training_database/ext_crypto_database/extended_db/1bsq_download_pdbs/6fxb.pdb C ../generate_training_database/gen_crypto_database/holo_structures/1gx8.pdb A

SALIGN_____> adding the next group to the alignment; iteration    1

SALIGN_____> adding the 

...


SALIGN_____> adding the next group to the alignment; iteration    1
num_passed 88 args ../generate_training_database/ext_crypto_database/extended_db/3puw_download_pdbs/5m14.pdb B ../generate_training_database/gen_crypto_database/holo_structures/1fqc.pdb A

SALIGN_____> adding the next group to the alignment; iteration    1

SALIGN_____> adding the next group to the alignment; iteration    1
num_passed 88 args ../generate_training_database/ext_crypto_database/extended_db/3puw_download_pdbs/4o4b.pdb A ../generate_training_database/gen_crypto_database/holo_structures/1fqc.pdb A

SALIGN_____> adding the next group to the alignment; iteration    1

SALIGN_____> adding the next group to the alignment; iteration    1
num_passed 88 args ../generate_training_database/ext_crypto_database/extended_db/3puw_download_pdbs/5aq9.pdb B ../generate_training_database/gen_crypto_database/holo_structures/1fqc.pdb A

SALIGN_____> adding the next group to the alignment; iteration    1

SALIGN_____> adding t


SALIGN_____> adding the next group to the alignment; iteration    1

SALIGN_____> adding the next group to the alignment; iteration    1
num_passed 88 args ../generate_training_database/ext_crypto_database/extended_db/3puw_download_pdbs/3puw.pdb E ../generate_training_database/gen_crypto_database/holo_structures/1fqc.pdb A

SALIGN_____> adding the next group to the alignment; iteration    1

SALIGN_____> adding the next group to the alignment; iteration    1
num_passed 88 args ../generate_training_database/ext_crypto_database/extended_db/3puw_download_pdbs/3pux.pdb E ../generate_training_database/gen_crypto_database/holo_structures/1fqc.pdb A

SALIGN_____> adding the next group to the alignment; iteration    1

SALIGN_____> adding the next group to the alignment; iteration    1
num_passed 88 args ../generate_training_database/ext_crypto_database/extended_db/3puw_download_pdbs/1peb.pdb A ../generate_training_database/gen_crypto_database/holo_structures/1fqc.pdb A

SALIGN_____> adding t


SALIGN_____> adding the next group to the alignment; iteration    1
num_passed 90 args ../generate_training_database/ext_crypto_database/extended_db/4hb2_download_pdbs/3m1i.pdb C ../generate_training_database/gen_crypto_database/holo_structures/4hat.pdb C

SALIGN_____> adding the next group to the alignment; iteration    1

SALIGN_____> adding the next group to the alignment; iteration    1
num_passed 90 args ../generate_training_database/ext_crypto_database/extended_db/4hb2_download_pdbs/5uwp.pdb C ../generate_training_database/gen_crypto_database/holo_structures/4hat.pdb C

SALIGN_____> adding the next group to the alignment; iteration    1

SALIGN_____> adding the next group to the alignment; iteration    1
num_passed 90 args ../generate_training_database/ext_crypto_database/extended_db/4hb2_download_pdbs/5dif.pdb C ../generate_training_database/gen_crypto_database/holo_structures/4hat.pdb C

SALIGN_____> adding the next group to the alignment; iteration    1

SALIGN_____> adding t


SALIGN_____> adding the next group to the alignment; iteration    1

SALIGN_____> adding the next group to the alignment; iteration    1
num_passed 90 args ../generate_training_database/ext_crypto_database/extended_db/4hb2_download_pdbs/5dh9.pdb C ../generate_training_database/gen_crypto_database/holo_structures/4hat.pdb C

SALIGN_____> adding the next group to the alignment; iteration    1

SALIGN_____> adding the next group to the alignment; iteration    1
num_passed 90 args ../generate_training_database/ext_crypto_database/extended_db/4hb2_download_pdbs/6a3c.pdb C ../generate_training_database/gen_crypto_database/holo_structures/4hat.pdb C

SALIGN_____> adding the next group to the alignment; iteration    1

SALIGN_____> adding the next group to the alignment; iteration    1
num_passed 90 args ../generate_training_database/ext_crypto_database/extended_db/4hb2_download_pdbs/6a3b.pdb C ../generate_training_database/gen_crypto_database/holo_structures/4hat.pdb C

SALIGN_____> adding t

,prot_id,chain_id,resnum,holo_prot_id,holo_chain_id,holo_resnum,crypto_apo_prot_id,crypto_apo_chain_id,crypto_apo_resnum,is_site
0,2iuz,B,39,2iuz,B,39,3che,A,39,False
1,3che,A,39,2iuz,B,39,3che,A,39,False
2,2iuz,B,40,2iuz,B,40,3che,A,40,False
3,3che,A,40,2iuz,B,40,3che,A,40,False
4,2iuz,B,41,2iuz,B,41,3che,A,41,False


## Find the Concavity score for each residue in df_train.
conc_scores_all stores the Concavity scores for each residue of each protein.  First the proteins in the original Cryptosite database are added.  Then the extended-database proteins are added.

In [2]:
num_passed = 0 # Useful for estimating how fast the code is running
conc_scores_all = {} # key is PDB_id__chain_id; value is apo-numbered conc score dict.

# Get the scores for the original Cryptosite database proteins.
df_cryptosite_orig = pd.read_csv("../generate_training_database/gen_crypto_database/cryptosite_database.csv")
for index, row in df_cryptosite_orig.iterrows():
    holo_pdb_id = row["holo_pdb_id"].lower()
    holo_pdb_loc = ("../generate_training_database/gen_crypto_database/holo_structures/%s.pdb" %(holo_pdb_id))
    holo_chain_id = row["holo_chain"]
    next_holo_conc_output_dir = "%s/holo_%s%s" %(conc_output_dir, holo_pdb_id, holo_chain_id)
    if "%s_%s" %(holo_pdb_id, holo_chain_id) not in conc_scores_all:
        conc_scores_this_holo = get_concavity_score(holo_pdb_loc, holo_chain_id, next_holo_conc_output_dir)
        conc_scores_all["%s_%s" %(holo_pdb_id, holo_chain_id)] = conc_scores_this_holo
        print("finding concavity score for holo", holo_pdb_id)
    crypto_apo_pdb_id = row["apo_pdb_id"].lower()
    crypto_apo_pdb_loc = ("../generate_training_database/gen_crypto_database/"
                          "apo_structures/%s.pdb" %(crypto_apo_pdb_id))
    crypto_apo_chain_id = row["apo_chain"]
    next_crypto_apo_conc_output_dir = "%s/crypto_apo_%s%s" %(conc_output_dir, crypto_apo_pdb_id, crypto_apo_chain_id)
    if "%s_%s" %(crypto_apo_pdb_id, crypto_apo_chain_id) not in conc_scores_all:
        conc_scores_this_crypto_apo = get_concavity_score(crypto_apo_pdb_loc, crypto_apo_chain_id, 
                                                          next_crypto_apo_conc_output_dir)
        conc_scores_all["%s_%s" %(crypto_apo_pdb_id, crypto_apo_chain_id)] = conc_scores_this_crypto_apo
        print("finding concavity score for crypto_apo", crypto_apo_pdb_id)

# Get the scores for the extended Cryptosite database proteins.
for apo_list_loc in sorted(glob.glob("../generate_training_database/ext_crypto_database/extended_db/*good*")):
    print("num_passed", num_passed)
    crypto_apo_pdb_id = apo_list_loc[62:66]
    crypto_apo_chain_id = dict_crypto_apo_chains[crypto_apo_pdb_id]
    holo_pdb_id = dict_crypto_apo_to_holo[crypto_apo_pdb_id]
    holo_chain_id = dict_holo_chains[holo_pdb_id]
    holo_pdb_loc = ("../generate_training_database/gen_crypto_database/holo_structures/%s.pdb" %(holo_pdb_id))
    
    next_conc_output_dir = "%s/holo_%s%s" %(conc_output_dir, holo_pdb_id, holo_chain_id)
    conc_scores_for_holo = get_concavity_score(holo_pdb_loc, holo_pdb_id, next_conc_output_dir)
    conc_scores_all["%s_%s" %(holo_pdb_id, holo_chain_id)] = conc_scores_for_holo
    print("finding conc scores for apos of holo %s_%s" %(holo_pdb_id, holo_chain_id))
    
    with open(apo_list_loc, "r") as apo_list_opened:
        apo_list_lines = apo_list_opened.readlines()
    for line in apo_list_lines[1:min(len(apo_list_lines), 51)]: # Skip the header, and read only 50 lines.
        apo_pdb_id = line[0:4]
        apo_pdb_loc = ("../generate_training_database/ext_crypto_database/extended_db/"
                       "%s_download_pdbs/%s.pdb" %(crypto_apo_pdb_id, apo_pdb_id))
        apo_chain_id = line[5]
        next_conc_output_dir = "%s/apo_%s%s_holo_%s%s" %(conc_output_dir, apo_pdb_id, apo_chain_id,
                                                         holo_pdb_id, holo_chain_id)
        conc_scores_apo_numbered = get_concavity_score(apo_pdb_loc, apo_pdb_id, next_conc_output_dir)
        conc_scores_all["%s_%s" %(apo_pdb_id, apo_chain_id)] = conc_scores_apo_numbered
    num_passed += 1

def lookup_conc_score(row):
    resnum_and_chain = "%s:%s" %(row["resnum"], row["chain_id"])
    if resnum_and_chain in conc_scores_all["%s_%s" %(row["prot_id"], row["chain_id"])]:
        return conc_scores_all["%s_%s" %(row["prot_id"], row["chain_id"])][resnum_and_chain]
    else:
        return 0

df_train["conc_score"] = df_train.apply(lookup_conc_score, axis=1)

finding concavity score for holo 2iuz
finding concavity score for crypto_apo 3che
finding concavity score for holo 1yv3
finding concavity score for crypto_apo 2aka
finding concavity score for holo 2jds
finding concavity score for crypto_apo 2gfc
finding concavity score for holo 1lic
finding concavity score for crypto_apo 1alb
finding concavity score for holo 2hka
finding concavity score for crypto_apo 1nep
finding concavity score for holo 3eks
finding concavity score for crypto_apo 3mn9
finding concavity score for holo 1nx3
finding concavity score for crypto_apo 1alv
finding concavity score for holo 2yqs
finding concavity score for crypto_apo 2yqc
finding concavity score for holo 2wi7
finding concavity score for crypto_apo 2qfo
finding concavity score for holo 1br6
finding concavity score for crypto_apo 1rtc
finding concavity score for holo 1j6z
finding concavity score for crypto_apo 1rdw
finding concavity score for holo 1tr5
finding concavity score for crypto_apo 1tqo
finding concavit

num_passed 13
finding conc scores for apos of holo 1ha3_B
num_passed 14
finding conc scores for apos of holo 1l5s_B
num_passed 15
finding conc scores for apos of holo 2oo8_X
num_passed 16
finding conc scores for apos of holo 3hl8_A
num_passed 17
finding conc scores for apos of holo 1gzf_C
num_passed 18
finding conc scores for apos of holo 1g67_B
num_passed 19
finding conc scores for apos of holo 2ixu_A
num_passed 20
finding conc scores for apos of holo 1ghy_H
num_passed 21
finding conc scores for apos of holo 3ip0_A
num_passed 22
finding conc scores for apos of holo 1cib_A
num_passed 23
finding conc scores for apos of holo 1imb_B
num_passed 24
finding conc scores for apos of holo 1wun_H
num_passed 25
finding conc scores for apos of holo 1pzo_A
num_passed 26
finding conc scores for apos of holo 1u1d_F
num_passed 27
finding conc scores for apos of holo 2egh_B
num_passed 28
finding conc scores for apos of holo 2ofp_A
num_passed 29
finding conc scores for apos of holo 1grn_A
num_passed 30


## For each residue, find the Concavity score for its neighbors on the sequence.
If the residue has 2 neighbors, then find the average between the neighbors.  If the residue has 1 neighbor, then find that score.  If the neighbors can't be found easily (ex. because the residue numbering has insertion codes), then set this feature to 0.

In [3]:
def lookup_neighbors_conc_score(row):
    apo_chain_id = row["chain_id"]
    apo_resnum = row["resnum"]
    
    # There are a few cases (ex. iloc[920]) where there is a valid crypto_apo_resnum, but
    # resnum is NaN.  In these cases, it's impossible to calculate the score.  
    # In other cases (ex. iloc 19231) apo_resnum may contain an insertion code (ex. 1H).
    # These cases are somewhat rare, and handling them well would take too much effort.  So I assigned
    # a value of 0 for these cases..
    try:
        apo_resnum_minus = str(int(apo_resnum) - 1)
    except ValueError:
        return 0
    apo_resnum_and_chain_minus = "%s:%s" %(apo_resnum_minus, apo_chain_id)
    if apo_resnum_and_chain_minus in conc_scores_all["%s_%s" %(row["prot_id"], row["chain_id"])]:
        has_minus = True
        conc_score_minus = conc_scores_all["%s_%s" %(row["prot_id"], row["chain_id"])][apo_resnum_and_chain_minus]
    else:
        has_minus = False
    
    apo_resnum_plus = str(int(apo_resnum) + 1)
    apo_resnum_and_chain_plus = "%s:%s" %(apo_resnum_plus, apo_chain_id)
    if apo_resnum_and_chain_plus in conc_scores_all["%s_%s" %(row["prot_id"], row["chain_id"])]:
        has_plus = True
        conc_score_plus = conc_scores_all["%s_%s" %(row["prot_id"], row["chain_id"])][apo_resnum_and_chain_plus]
    else:
        has_plus = False
    
    if has_plus and (not has_minus):
        return conc_score_plus
    elif has_minus and (not has_plus):
        return conc_score_minus
    elif (not has_minus) and (not has_plus):
        return 0
    else:
        return ((conc_score_plus + conc_score_minus) / 2)

df_train["neighbors_conc_score"] = df_train.apply(lookup_neighbors_conc_score, axis=1)

## Find the difference between the Concavity scores of each protein and the scores of the corresponding apo protein from the original Cryptosite database.
In theory, the difference should be low for apo proteins because there isn't a pocket in those cases.  The difference should be higher for pocket residues in holo structures.  The actual trend may be more complex than this because of the limitations of Concavity.  The ML algorithm is hopefully able to figure out what's going on.

In [4]:
def get_delta_conc_score(row):
    apo_resnum_and_chain = "%s:%s" %(row["resnum"], row["chain_id"])
    if apo_resnum_and_chain in conc_scores_all["%s_%s" %(row["prot_id"], row["chain_id"])]:
        apo_conc_score = conc_scores_all["%s_%s" %(row["prot_id"], row["chain_id"])][apo_resnum_and_chain]
        crypto_apo_resnum_and_chain = "%s:%s" %(row["crypto_apo_resnum"], row["crypto_apo_chain_id"])
        crypto_apo_conc_dict = conc_scores_all["%s_%s" %(row["crypto_apo_prot_id"], row["crypto_apo_chain_id"])]
        if crypto_apo_resnum_and_chain in crypto_apo_conc_dict:
            crypto_apo_conc_score = crypto_apo_conc_dict[crypto_apo_resnum_and_chain]
            return apo_conc_score - crypto_apo_conc_score
    return 0

df_train["delta_conc_score"] = df_train.apply(get_delta_conc_score, axis=1)

## Find the difference between Concavity scores (current - cryptosite apo) for neighboring residues.
Neighbors are found as described above.

In [5]:
def get_neighbors_delta_conc_score(row):
    apo_pdb_id = row["prot_id"]
    apo_chain_id = row["chain_id"]
    apo_resnum = row["resnum"]
    crypto_apo_pdb_id = row["crypto_apo_prot_id"]
    crypto_apo_chain_id = row["crypto_apo_chain_id"]
    crypto_apo_resnum = row["crypto_apo_resnum"]
    
    # There are a few cases (ex. iloc[920]) where there is a valid crypto_apo_resnum, but
    # resnum is NaN.  In these cases, it's impossible to calculate the score.  
    # In other cases (ex. iloc 19231) apo_resnum may contain an insertion code (ex. 1H).
    # These cases are somewhat rare, and handling them well would take too much effort.  So I assigned
    # a value of 0 for these cases..
    try:
        apo_resnum_minus = str(int(apo_resnum) - 1)
    except ValueError:
        return 0
    apo_resnum_and_chain_minus = "%s:%s" %(apo_resnum_minus, apo_chain_id)
    has_minus = False # Initialization; it may be set to True later.
    if apo_resnum_and_chain_minus in conc_scores_all["%s_%s" %(apo_pdb_id, apo_chain_id)]:
        apo_conc_score_minus = conc_scores_all["%s_%s" %(apo_pdb_id, apo_chain_id)][apo_resnum_and_chain_minus]
        # crypto_apo_resnum might be NaN; return 0 when this happens.
        try:
            crypto_apo_resnum_minus = str(int(crypto_apo_resnum) - 1)
        except ValueError:
            return 0
        crypto_apo_resnum_and_chain_minus = "%s:%s" %(crypto_apo_resnum_minus, crypto_apo_chain_id)
        crypto_apo_conc_dict = conc_scores_all["%s_%s" %(crypto_apo_pdb_id, crypto_apo_chain_id)]
        if crypto_apo_resnum_and_chain_minus in crypto_apo_conc_dict:
            crypto_apo_conc_score_minus = crypto_apo_conc_dict[crypto_apo_resnum_and_chain_minus]
            has_minus = True
            delta_conc_score_minus = apo_conc_score_minus - crypto_apo_conc_score_minus
            
    apo_resnum_plus = str(int(apo_resnum) + 1)
    apo_resnum_and_chain_plus = "%s:%s" %(apo_resnum_plus, apo_chain_id)
    has_plus = False # Initialization; it may be set to True later.
    if apo_resnum_and_chain_plus in conc_scores_all["%s_%s" %(apo_pdb_id, apo_chain_id)]:
        apo_conc_score_plus = conc_scores_all["%s_%s" %(apo_pdb_id, apo_chain_id)][apo_resnum_and_chain_plus]
        try:
            crypto_apo_resnum_plus = str(int(crypto_apo_resnum) + 1)
        except ValueError:
            return 0
        crypto_apo_resnum_and_chain_plus = "%s:%s" %(crypto_apo_resnum_plus, crypto_apo_chain_id)
        crypto_apo_conc_dict = conc_scores_all["%s_%s" %(crypto_apo_pdb_id, crypto_apo_chain_id)]
        if crypto_apo_resnum_and_chain_plus in crypto_apo_conc_dict:
            crypto_apo_conc_score_plus = crypto_apo_conc_dict[crypto_apo_resnum_and_chain_plus]
            has_plus = True
            delta_conc_score_plus = apo_conc_score_plus - crypto_apo_conc_score_plus
    if has_plus and (not has_minus):
        return delta_conc_score_plus
    elif has_minus and (not has_plus):
        return delta_conc_score_minus
    elif (not has_minus) and (not has_plus):
        return 0
    else:
        return ((delta_conc_score_plus + delta_conc_score_minus) / 2)

df_train["neighbors_delta_conc_score"] = df_train.apply(get_neighbors_delta_conc_score, axis=1)

## Find the difference in alpha-carbon position between Cryptosite apo and current structure.

In [6]:
import os
import math

ca_pos_dict = {}
def get_ca_pos(pdb_loc, chain_id, resnum):
    all_info = "%s:%s:%s" %(pdb_loc, chain_id, resnum)
    if all_info in ca_pos_dict:
        return ca_pos_dict[all_info]
    resnum = str(resnum) # So that it can be compared with resnums containing insertion codes.
    with open(pdb_loc, "r") as pdb_file:
        pdb_file_lines = pdb_file.readlines()
    for line in pdb_file_lines:
        if ((len(line) >= 21) and (line[0:4] == "ATOM") and (line[21] == chain_id) and
            (line[22:27].strip() == resnum) and (line[13:15] == "CA")):
            x_pos = float(line[30:38])
            y_pos = float(line[38:46])
            z_pos = float(line[46:54])
            ca_pos_dict[all_info] = [x_pos, y_pos, z_pos]
            return [x_pos, y_pos, z_pos]
    ca_pos_dict[all_info] = None
    return None
    

def get_delta_pos(row):
    apo_pdb_id = row["prot_id"]
    apo_chain_id = row["chain_id"]
    apo_resnum = row["resnum"]
    holo_pdb_id = row["holo_prot_id"]
    holo_chain_id = row["holo_chain_id"]
    holo_resnum = row["holo_resnum"]
    crypto_apo_pdb_id = row["crypto_apo_prot_id"]
    crypto_apo_chain_id = row["crypto_apo_chain_id"]
    crypto_apo_resnum = row["crypto_apo_resnum"]
    
    # The PDB file is probably in extended_db.  But there are a few exceptions, eg. see the discussion of 1AFQ
    # above.  So check multiple places where the file might be.
    #
    # In this code, the protein being compared to the cryptosite_apo protein is called the "apo" protein.
    # This protein is usually apo, except for the cryptosite_holo protein.  Thus loc_3 isn't a mistake, even
    # though it looks weird to look for apo_pdb_id.pdb in holo_structures.
    loc_1 = ("../generate_training_database/ext_crypto_database/extended_db/"
             "%s_download_pdbs/%s.pdb" %(crypto_apo_pdb_id, apo_pdb_id))
    loc_2 = ("../generate_training_database/gen_crypto_database/"
             "apo_structures/%s.pdb" %(apo_pdb_id))
    loc_3 = ("../generate_training_database/gen_crypto_database/"
             "holo_structures/%s.pdb" %(apo_pdb_id))
    if os.path.exists(loc_1):
        apo_pdb_loc = loc_1
    elif os.path.exists(loc_2):
        apo_pdb_loc = loc_2
    else:
        apo_pdb_loc = loc_3

    crypto_apo_pdb_loc = ("../generate_training_database/gen_crypto_database/"
                          "apo_structures/%s.pdb" %(crypto_apo_pdb_id))
    apo_pos = get_ca_pos(apo_pdb_loc, apo_chain_id, apo_resnum)
    crypto_apo_pos = get_ca_pos(crypto_apo_pdb_loc, crypto_apo_chain_id, crypto_apo_resnum)
    if apo_pos is None or crypto_apo_pos is None:
        return 0
    else:
        return math.sqrt((apo_pos[0] - crypto_apo_pos[0])**2 + (apo_pos[1] - crypto_apo_pos[1])**2 +
                         (apo_pos[2] - crypto_apo_pos[2])**2)

df_train["delta_pos"] = df_train.apply(get_delta_pos, axis=1)

## Find the difference in alpha-carbon position of neighbors.

In [7]:
def get_neighbors_delta_pos(row):
    apo_pdb_id = row["prot_id"]
    apo_chain_id = row["chain_id"]
    apo_resnum = row["resnum"]
    
    holo_pdb_id = row["holo_prot_id"]
    holo_chain_id = row["holo_chain_id"]
    holo_resnum = row["holo_resnum"]
    
    crypto_apo_pdb_id = row["crypto_apo_prot_id"]
    crypto_apo_chain_id = row["crypto_apo_chain_id"]
    crypto_apo_resnum = row["crypto_apo_resnum"]
    
    # See above for an explanation of this.
    loc_1 = ("../generate_training_database/ext_crypto_database/extended_db/"
             "%s_download_pdbs/%s.pdb" %(crypto_apo_pdb_id, apo_pdb_id))
    loc_2 = ("../generate_training_database/gen_crypto_database/"
             "apo_structures/%s.pdb" %(apo_pdb_id))
    loc_3 = ("../generate_training_database/gen_crypto_database/"
             "holo_structures/%s.pdb" %(apo_pdb_id))
    if os.path.exists(loc_1):
        apo_pdb_loc = loc_1
    elif os.path.exists(loc_2):
        apo_pdb_loc = loc_2
    else:
        apo_pdb_loc = loc_3

    crypto_apo_pdb_loc = ("../generate_training_database/gen_crypto_database/"
                          "apo_structures/%s.pdb" %(crypto_apo_pdb_id))
    try:
        apo_resnum_minus = str(int(apo_resnum) - 1)
    except ValueError:
        return 0
    apo_pos_minus = get_ca_pos(apo_pdb_loc, apo_chain_id, apo_resnum_minus)
    try:
        crypto_apo_resnum_minus = str(int(crypto_apo_resnum) - 1)
    except ValueError:
        return 0
    crypto_apo_pos_minus = get_ca_pos(crypto_apo_pdb_loc, crypto_apo_chain_id, crypto_apo_resnum_minus)
    if (apo_pos_minus is None) or (crypto_apo_pos_minus is None):
        has_minus = False
    else:
        has_minus = True
        delta_minus = math.sqrt((apo_pos_minus[0] - crypto_apo_pos_minus[0])**2 + 
                                (apo_pos_minus[1] - crypto_apo_pos_minus[1])**2 +
                                (apo_pos_minus[2] - crypto_apo_pos_minus[2])**2)
    
    apo_resnum_plus = str(int(apo_resnum) + 1)
    apo_pos_plus = get_ca_pos(apo_pdb_loc, apo_chain_id, apo_resnum_plus)
    crypto_apo_resnum_plus = str(int(crypto_apo_resnum) + 1)
    crypto_apo_pos_plus = get_ca_pos(crypto_apo_pdb_loc, crypto_apo_chain_id, crypto_apo_resnum_plus)
    if (apo_pos_plus is None) or (crypto_apo_pos_plus is None):
        has_plus = False
    else:
        has_plus = True
        delta_plus = math.sqrt((apo_pos_plus[0] - crypto_apo_pos_plus[0])**2 +
                               (apo_pos_plus[1] - crypto_apo_pos_plus[1])**2 +
                               (apo_pos_plus[2] - crypto_apo_pos_plus[2])**2)
    if has_plus and (not has_minus):
        return delta_plus
    elif has_minus and (not has_plus):
        return delta_minus
    elif (not has_minus) and (not has_plus):
        return 0
    else:
        return ((delta_plus + delta_minus) / 2)
df_train["neighbors_delta_pos"] = df_train.apply(get_neighbors_delta_pos, axis=1)

## All the features have been calculated.  Here is the dataframe.

In [8]:
df_train.head()

,prot_id,chain_id,resnum,holo_prot_id,holo_chain_id,holo_resnum,crypto_apo_prot_id,crypto_apo_chain_id,crypto_apo_resnum,is_site,conc_score,neighbors_conc_score,delta_conc_score,neighbors_delta_conc_score,delta_pos,neighbors_delta_pos
0,2iuz,B,39,2iuz,B,39,3che,A,39,False,0.0,0.0,0.0,0.0,48.643946,54.636592
1,3che,A,39,2iuz,B,39,3che,A,39,False,0.0,0.0,0.0,0.0,0.000000,0.000000
2,2iuz,B,40,2iuz,B,40,3che,A,40,False,0.0,0.0,0.0,0.0,54.636592,54.545598
3,3che,A,40,2iuz,B,40,3che,A,40,False,0.0,0.0,0.0,0.0,0.000000,0.000000
4,2iuz,B,41,2iuz,B,41,3che,A,41,False,0.0,0.0,0.0,0.0,60.447251,58.324066


## Estimate the model's effectiveness.
The data is divided into a training set and a test set.  5-fold cross validation is used on the training set to get the best values of hyperparameters.  Then the trained model is applied to the test set.  The results are used to estimate the model's effectiveness.

In [9]:
import numpy as np
from sklearn.model_selection import GridSearchCV, KFold, cross_val_score, cross_validate, train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn import preprocessing

cv_object = KFold(n_splits=3, shuffle=True, random_state=0)
rf_clf = RandomForestClassifier(class_weight="balanced", random_state=0)

param_grid = {"max_depth":np.arange(1,12,4), "n_estimators":[10, 50],
              "max_features":[0.1, 0.5, 0.7, 1, "auto"]}

rf_grid_search = GridSearchCV(rf_clf, param_grid=param_grid, cv=cv_object, iid=False, scoring="roc_auc")

pipeline = Pipeline([("scaler", preprocessing.StandardScaler()), ("rf_grid_search", rf_grid_search)])

included_columns = ["conc_score", "neighbors_conc_score", "delta_conc_score", "neighbors_delta_conc_score",
                    "delta_pos", "neighbors_delta_pos"]
X = df_train[included_columns]
y = df_train["is_site"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=1, shuffle=True)
pipeline.fit(X_train, y_train)

y_pred = pipeline.predict(X_test)
y_true = y_test.values

## Display the confusion matrix.

In [10]:
pd.crosstab(y_true, y_pred, rownames=["Actual"], colnames=["Predicted"], margins=True)

Predicted,False,True,All
Actual,,,
False,124876,22539,147415
True,87,229,316
All,124963,22768,147731


## Calculate the AUROC using the results of the predictions on the test set.

In [11]:
from sklearn import metrics
y_scores = pipeline.predict_proba(X_test)
scores_pos = []
for score in y_scores:
    scores_pos.append(score[1])
metrics.roc_auc_score(y_true, scores_pos)

0.8558173837143652

## Get the feature importances.

In [12]:
pipeline.steps[1][1].best_estimator_.feature_importances_

array([0.58558474, 0.07652541, 0.18603476, 0.01940393, 0.04148142,
       0.09096973])

## Train the model on the entire dataset.  Save the model.
Separating the data into training and test sets is necessary for estimating model effectiveness.  But it makes the model slightly worse because it reduces the number of data points that it is trained on.  To fix this, retrain the model on a combination of training and test sets.

Save the model so that it can be used by other Python code.

In [13]:
import datetime
from joblib import dump

pipeline.fit(X, y)

toBePersisted = dict({
    'model': pipeline,
    'metadata': {
        'name': 'Cryptic Site in MD',
        'author': 'Daniel Evans',
        'date': datetime.datetime.now().strftime("%B %d %Y"),
    }
})

dump(toBePersisted, 'trained_model.joblib')

['trained_model.joblib']